In [1]:
import pandas as pd
import numpy as np
import sklearn as sk

In [ ]:
#pegar so os filmes mais recorrentes
#indicar apenas com rmse abaixo de 0.5
#caso nao encontrar 3 vizinhos, dividir os filmes em 75%, senao novamente, em 50% (esses devem ser aleatorios ou os mais recentes?, talvez seja necessario ordenalos por timestamp)
#dar a opçao por content-like ou collaborative filtering
#caso nao encontrar 3 vizinhos, pegar os 75% de filmes do usuario que sao mais famosos

In [13]:
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import csr_matrix

In [2]:
df = pd.read_csv('ml-latest-small/ratings.csv')

In [15]:
df

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [ ]:
popularity = (
    df.groupby("movieId")          # agrupa por filme
      .size()                      # conta avaliações por filme
      .rename("popularity")        # renomeia a série
      .reset_index()               # transforma o índice (movieId) em coluna
      .sort_values("popularity", ascending=False)  # opcional: mais populares primeiro
)

print(popularity.head())



In [ ]:
pop = popularity.iloc[:20000,:]

In [ ]:
pop['popularity'].sum()

In [ ]:
popularity.to_csv('popularity.csv', index=False)

In [ ]:
df.isna().sum()

In [ ]:
df = df.iloc[:3000000, :]

In [ ]:
popularity = (
    df.groupby("movieId")          # agrupa por filme
      .size()                      # conta avaliações por filme
      .rename("popularity")        # renomeia a série
      .reset_index()               # transforma o índice (movieId) em coluna
      .sort_values("popularity", ascending=False)  # opcional: mais populares primeiro
)

print(popularity.head())

In [ ]:
popularity.iloc[:]

In [16]:
df_pivot = df.pivot_table(index='userId', columns='movieId', values='rating', aggfunc='mean')

In [ ]:
df_pivot

In [ ]:
df_pivot

In [3]:
df_pivot = pd.read_csv("df_pivot.csv")

In [5]:
df_pivot.columns

Index(['userId', '1', '2', '3', '4', '5', '6', '7', '8', '9',
       ...
       '193565', '193567', '193571', '193573', '193579', '193581', '193583',
       '193585', '193587', '193609'],
      dtype='object', length=9725)

In [ ]:
import pandas as pd

def exportar_popularidade(df_pivot, caminho_csv="popularidade_filmes.csv"):
    """
    Cria um CSV com movieId e quantidade de avaliações (do mais popular ao menos).
    """
    # Conta quantos usuários avaliaram cada filme (não NaN)
    popularidade = df_pivot.notna().sum().sort_values(ascending=False)

    # Transforma em DataFrame
    df_popularidade = pd.DataFrame({
        "movieId": popularidade.index,
        "quantidade_avaliacoes": popularidade.values
    })

    # Salva em CSV
    df_popularidade.to_csv(caminho_csv, index=False, encoding="utf-8")
    print(f"📂 Arquivo salvo em: {caminho_csv}")

    return df_popularidade


# Exemplo de uso:
df_pop = exportar_popularidade(df_pivot, "popularidade_filmes.csv")
df_pop.head()


In [ ]:
from scipy.sparse import csr_matrix
knn = NearestNeighbors(metric='cosine', algorithm='brute')

for idx, row in df_pivot.iterrows():
    # Selecionar apenas as colunas com avaliações (sem NaN)
    row_valid = row.dropna()
    
    # Se houver menos de 2 filmes avaliados, pula
    if len(row_valid) < 3:
        continue

    # Filmes avaliados por esse usuário
    filmes = row_valid.index.tolist()

    # Filtrar DataFrame com apenas esses filmes
    validos = df_pivot[filmes].dropna(how='any')  # Apenas usuários que avaliaram TODOS esses filmes
    
    # Remover o usuário atual
    validos = validos.drop(index=idx, errors='ignore')
    
    # Se não houver outros usuários com essas mesmas avaliações, pula
    if validos.shape[0] == 0:
        continue

    # Criar matriz esparsa
    matrix_sparse = csr_matrix(validos.values)
    user_vector = csr_matrix(row_valid.values.reshape(1, -1))
    if matrix_sparse.shape[0] >= 3:
        knn.fit(matrix_sparse)
        distances, indices = knn.kneighbors(user_vector, n_neighbors=3, return_distance=True)
        distance = np.array(distances[0])
        print(f' {distance} - vizinhos: {indices.flatten()}')
       
       
    else:
        print(f'Usuário {idx} ignorado - não há amostras suficientes para KNN')
    # Treinar modelo
        knn.fit(matrix_sparse)


In [6]:
ratings = {1:3.0, 260:5.0, 296:5.0, 318:3.0, 356:3.5, 480:4.5, 527:3.0, 589:4.5, 2959:4.0}
#[1136, 48385, 608]

In [ ]:
df_pivot

In [ ]:
df_pivot = df_pivot.astype(np.float32)
df_pivot.columns = df_pivot.columns.astype(str)
df_pivot.to_parquet("df_pivot.parquet", engine="fastparquet", index=True)


In [ ]:

df_pivot = df_pivot.fillna(0)

# 3. Converter para a matriz esparsa usando csr_matrix
# O .values converte o DataFrame para uma matriz NumPy antes de criar a matriz esparsa
df_pivot_esparso = csr_matrix(df_pivot.values)

In [18]:
df_pivot.to_csv("df_pivot.csv", index=True)

In [19]:
df_pivot

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,NaN,4.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,2.5,NaN,NaN,NaN,NaN,NaN,2.5,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
607,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
608,2.5,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
popularidade = df_pivot.notna().sum().sort_values(ascending=False)

In [21]:
popularidade = pd.DataFrame({
    "movieId": popularidade.index,
    "popularidade": popularidade.values
})

In [22]:
popularidade.to_csv('popularidade_aplicacao.csv', index=True)

In [11]:
def recomendar_usuario(df_pivot, meus_ratings, limiar_sim=0.7, random_state=42, caminho_popularidade="popularidade_aplicacao.csv"):
    # ... (cole sua função recomendar_usuario aqui) ...
    np.random.seed(random_state)
    minha_linha = pd.Series(meus_ratings, name='meu_usuario')
    meus_filmes = list(meus_ratings.keys())
    
    df_pop = pd.read_csv(caminho_popularidade)
    popularidade = df_pop.set_index("movieId")["popularidade"].to_dict()
    
    # ... (o restante da sua função) ...
    estrategias = [
        (1.0, 'todos'),
        (0.75, 'aleatorio'),
        (0.75, 'populares'),
        (0.5, 'aleatorio'),
        (0.5, 'populares'),
        (0.25, 'aleatorio'),
        (0.25, 'populares')
    ]
    
    melhor_usuario = None
    melhor_sim = -1
    vizinhos_extra = []

    for perc, modo in estrategias:
        print(f"\n🔎 Testando estratégia: {modo.upper()} ({int(perc*100)}% dos filmes)")

        if perc < 1.0:
            n_filmes = max(1, int(len(meus_filmes) * perc))
            if modo == 'aleatorio':
                filmes_selecionados = list(np.random.choice(meus_filmes, size=n_filmes, replace=False))
            elif modo == 'populares':
                filmes_ordenados = [f for f in meus_filmes if f in popularidade]
                filmes_ordenados.sort(key=lambda x: popularidade[x], reverse=True)
                filmes_selecionados = filmes_ordenados[:n_filmes]
        else:
            filmes_selecionados = meus_filmes
        
        print(f"🎬 Filmes usados nesta rodada: {filmes_selecionados}")

        df_filtrado = df_pivot.dropna(subset=filmes_selecionados, how='any')
        print(f"📊 Usuários restantes após filtro: {df_filtrado.shape[0]}")
        
        if df_filtrado.shape[0] == 0:
            print("⚠️ Nenhum usuário encontrado para essa seleção de filmes.")
            continue
        
        X = df_filtrado[filmes_selecionados].values
        minha_vetor = minha_linha[filmes_selecionados].values.reshape(1, -1)
        
        n_vizinhos = min(4, df_filtrado.shape[0]) 
        knn = NearestNeighbors(n_neighbors=n_vizinhos, metric='cosine')
        knn.fit(X)
        dist, ind = knn.kneighbors(minha_vetor, n_neighbors=n_vizinhos)
        
        dist = dist[0]
        ind = ind[0]
        
        sims = 1 - dist
        usuarios = df_filtrado.index[ind].tolist()
        
        print("👥 Vizinhos encontrados:")
        for u, s in zip(usuarios, sims):
            print(f"   - {u} (similaridade={s:.4f})")
        
        sim = sims[0]
        usuario_encontrado = usuarios[0]
        
        if sim > limiar_sim:
            print("✅ Similaridade acima do limiar, parando busca.")
            melhor_usuario = usuario_encontrado
            melhor_sim = sim
            vizinhos_extra = list(zip(usuarios[1:], sims[1:]))
            break
        else:
            print("❌ Similaridade baixa, tentando próxima estratégia...")

    return {
        'usuario_mais_similar': melhor_usuario,
        'similaridade': melhor_sim,
        'filmes_usados': meus_filmes,
        'vizinhos_extras': vizinhos_extra
    }

In [8]:
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors


def recomendar_usuario(df_pivot, meus_ratings, limiar_sim=0.7, random_state=42, caminho_popularidade="popularidade_aplicacao.csv"):
    """
    df_pivot: DataFrame pivotado userId x movieId com ratings (1-5, NaN se não avaliou)
    meus_ratings: dict {movieId: rating}
    limiar_sim: float, limiar mínimo de similaridade cosine
    caminho_popularidade: str, caminho para o CSV com colunas [movieId, popularidade]
    """
    np.random.seed(random_state)
    
    # Cria a linha do usuário
    minha_linha = pd.Series(meus_ratings, name='meu_usuario')
    
    # Lista de IDs dos filmes que forneceu
    meus_filmes = list(meus_ratings.keys())
    
    # 🔹 Lê o CSV de popularidade
    df_pop = pd.read_csv(caminho_popularidade)
    popularidade = df_pop.set_index("movieId")["popularidade"].to_dict()
    
    # Estratégias de filtragem: (percentual de filmes, modo)
    estrategias = [
        (1.0, 'todos'),
        (0.75, 'aleatorio'),
        (0.75, 'populares'),
        (0.5, 'aleatorio'),
        (0.5, 'populares'),
        (0.25, 'aleatorio'),
        (0.25, 'populares')
    ]
    
    melhor_usuario = None
    melhor_sim = -1
    vizinhos_extra = []

    for perc, modo in estrategias:
        print(f"\n🔎 Testando estratégia: {modo.upper()} ({int(perc*100)}% dos filmes)")

        # Escolher filmes para a rodada
        if perc < 1.0:
            n_filmes = max(1, int(len(meus_filmes) * perc))
            if modo == 'aleatorio':
                filmes_selecionados = list(np.random.choice(meus_filmes, size=n_filmes, replace=False))
            elif modo == 'populares':
                filmes_ordenados = [f for f in meus_filmes if f in popularidade]
                filmes_ordenados.sort(key=lambda x: popularidade[x], reverse=True)
                filmes_selecionados = filmes_ordenados[:n_filmes]
        else:
            filmes_selecionados = meus_filmes
        
        print(f"🎬 Filmes usados nesta rodada: {filmes_selecionados}")

        # Filtra usuários que avaliaram todos os filmes selecionados
        df_filtrado = df_pivot.dropna(subset=filmes_selecionados, how='any')
        print(f"📊 Usuários restantes após filtro: {df_filtrado.shape[0]}")
        
        if df_filtrado.shape[0] == 0:
            print("⚠️ Nenhum usuário encontrado para essa seleção de filmes.")
            continue
        
        # Construir matriz para cálculo de similaridade
        X = df_filtrado[filmes_selecionados].values
        minha_vetor = minha_linha[filmes_selecionados].values.reshape(1, -1)
        
        # KNN cosine (pegar até 4 vizinhos: 1 principal + 3 extras)
        n_vizinhos = min(4, df_filtrado.shape[0])  
        knn = NearestNeighbors(n_neighbors=n_vizinhos, metric='cosine')
        knn.fit(X)
        dist, ind = knn.kneighbors(minha_vetor, n_neighbors=n_vizinhos)
        
        dist = dist[0]
        ind = ind[0]
        
        # Calcular similaridades
        sims = 1 - dist
        usuarios = df_filtrado.index[ind].tolist()
        
        print("👥 Vizinhos encontrados:")
        for u, s in zip(usuarios, sims):
            print(f"   - {u} (similaridade={s:.4f})")
        
        # Melhor usuário (primeiro da lista)
        sim = sims[0]
        usuario_encontrado = usuarios[0]
        
        if sim > limiar_sim:
            print("✅ Similaridade acima do limiar, parando busca.")
            melhor_usuario = usuario_encontrado
            melhor_sim = sim
            vizinhos_extra = list(zip(usuarios[1:], sims[1:]))  # próximos 3
            break
        else:
            print("❌ Similaridade baixa, tentando próxima estratégia...")

    return {
        'usuario_mais_similar': melhor_usuario,
        'similaridade': melhor_sim,
        'filmes_usados': meus_filmes,
        'vizinhos_extras': vizinhos_extra  # lista de (usuario, similaridade)
    }




In [12]:
def recomendar_filmes(df_pivot, resultado_recomendar, meus_ratings, top_n=3, min_rating=4):
    """
    df_pivot: DataFrame pivotado userId x movieId com ratings
    resultado_recomendar: dict retornado pela função recomendar_usuario
    meus_ratings: dict {movieId: rating} -> filmes já assistidos pelo usuário
    top_n: quantidade de filmes a recomendar
    min_rating: nota mínima para recomendar
    
    Retorna: lista de movieIds recomendados
    """
    candidatos = [resultado_recomendar["usuario_mais_similar"]] + resultado_recomendar.get("vizinhos_extras", [])
    filmes_assistidos = set(meus_ratings.keys())
    
    for user_id in candidatos:
        if user_id is None:
            continue
        
        print(f"\n🎯 Tentando recomendar filmes do usuário {user_id}...")
        linha_usuario = df_pivot.loc[user_id]
        
        # Ordena os filmes avaliados pelo usuário (maior rating primeiro)
        filmes_ordenados = (
            linha_usuario.dropna()
            .sort_values(ascending=False)
        )
        
        # Filtra apenas filmes não assistidos e com rating >= min_rating
        filmes_validos = [mid for mid, nota in filmes_ordenados.items() 
                          if mid not in filmes_assistidos and nota >= min_rating]
        
        if len(filmes_validos) >= top_n:
            recomendados = filmes_validos[:top_n]
            print(f"✅ Filmes recomendados: {recomendados}")
            return recomendados
        else:
            print(f"⚠️ Usuário {user_id} não tem filmes suficientes (>= {min_rating}) para recomendar.")
    
    print("❌ Nenhum usuário tinha recomendações válidas.")
    return []


In [13]:
# Primeiro encontra o usuário mais próximo
resultado = recomendar_usuario(df_pivot, ratings, limiar_sim=0.7)

# Depois busca os filmes recomendados
recomendados = recomendar_filmes(df_pivot, resultado, ratings, top_n=3, min_rating=4)

print("🎬 Recomendação final:", recomendados)



🔎 Testando estratégia: TODOS (100% dos filmes)
🎬 Filmes usados nesta rodada: [1, 260, 296, 318, 356, 480, 527, 589, 2959]


KeyError: [1, 260, 296, 318, 356, 480, 527, 589, 2959]

In [ ]:
df_pivot.columns

In [23]:
import pandas as pd

# Lê os CSVs
movies = pd.read_csv("ml-latest-small/movies.csv")  # contém movieId e title
popularity = pd.read_csv("popularidade_aplicacao.csv")  # contém movieId

# Faz o merge (inner join) usando o movieId
filmes_usados = pd.merge(popularity, movies, on="movieId", how="inner")

# Salva o resultado
filmes_usados.to_csv("filmes_usados.csv", index=False)

print("CSV 'filmes_usados.csv' criado com sucesso!")


CSV 'filmes_usados.csv' criado com sucesso!


In [ ]:
from scipy.sparse import csr_matrix
import numpy as np
knn = NearestNeighbors(metric='euclidean', algorithm='brute')

# Define a threshold para RMSE (por exemplo, 1.0)
rmse_threshold = 1.0

for idx, row in df_pivot.iterrows():
    # Selecionar apenas as colunas com avaliações (sem NaN)
    row_valid = row.dropna()
    
    # Se houver menos de 3 filmes avaliados, pula
    if len(row_valid) < 3:
        continue

    # Filmes avaliados por esse usuário
    filmes = row_valid.index.tolist()

    # Filtrar DataFrame com apenas esses filmes
    validos = df_pivot[filmes].dropna(how='any')  # Apenas usuários que avaliaram TODOS esses filmes
    
    # Remover o usuário atual
    validos = validos.drop(index=idx, errors='ignore')
    
    # Se não houver outros usuários com essas mesmas avaliações, pula
    if validos.shape[0] == 0:
        continue

    # Criar matriz esparsa
    matrix_sparse = csr_matrix(validos.values)
    user_vector = csr_matrix(row_valid.values.reshape(1, -1))

    if matrix_sparse.shape[0] >= 3:
        knn.fit(matrix_sparse)
        distances, indices = knn.kneighbors(user_vector, n_neighbors=3, return_distance=True)
        
        # Calcular a distância Euclidiana
        distance = np.array(distances[0])
        
        # Normalizar a distância Euclidiana por k (número de filmes em comum)
        k = len(filmes)  # Número de filmes avaliados
        rmse_per_item = distance / np.sqrt(k)

        # Imprimir as distâncias e RMSE
        

        # Filtrar vizinhos com base no threshold de RMSE
        valid_neighbors = [i for i, rmse in zip(indices.flatten(), rmse_per_item) if rmse <= rmse_threshold]
        
        if valid_neighbors:
            print(f'Usuário {idx} tem vizinhos válidos (RMSE <= {rmse_threshold}): {valid_neighbors}')
        else:
            print(f'Usuário {idx} não tem vizinhos válidos com RMSE <= {rmse_threshold}')
        
    else:
        print(f'Usuário {idx} ignorado - não há amostras suficientes para KNN')

